# 🕳️ Pothole Detection Training with YOLOv8

This notebook trains a YOLOv8 model to detect potholes using a dataset from Roboflow/Kaggle.
**Runtime**: Make sure to set Runtime > Change runtime type > Hardware accelerator to **T4 GPU**.

In [1]:
# 1. Install Ultralytics YOLOv8
!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics 8.4.0 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 38.7/112.6 GB disk)


## 2. Download Dataset
Downloading the 'pothole-detection-2' dataset from Roboflow using your key.

In [2]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="7KW7Fth9EOjdB5UTxewS")
project = rf.workspace("pothole-detection-2").project("pothole-detection-2-zvaru")
version = project.version(6)
dataset = version.download("yolov8")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 74.1 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to pothole-detection-2-6 in yolov8:: 100%|██████████| 28688/28688 [00:05<00:00, 4885.21it/s]


## 3. Train Model
We use `yolov8n.pt` (Nano) for the fastest inference speed on edge/web deployments.

In [3]:
%cd /content
from ultralytics import YOLO

# Load model
model = YOLO('yolov8n.pt')  # load a pretrained model

# Train
# Find the data.yaml file automatically
import os

data_yaml_path = None
for root, dirs, files in os.walk("/content"):
    if "data.yaml" in files and "pothole" in root.lower():
        data_yaml_path = os.path.join(root, "data.yaml")
        break

# Fallback search if 'pothole' folder name varies
if not data_yaml_path:
     for root, dirs, files in os.walk("/content"):
        if "data.yaml" in files:
            data_yaml_path = os.path.join(root, "data.yaml")
            break

if data_yaml_path:
    print(f"Found data.yaml at: {data_yaml_path}")
    model.train(data=data_yaml_path, epochs=20, imgsz=640)
else:
    print("Could not find data.yaml. Please check the dataset folder manually.")

/content
Found data.yaml at: /content/pothole-detection-2-6/data.yaml
Ultralytics 8.4.0 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/pothole-detection-2-6/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=train, nbs=64, nms=False, opset=None, optimize=Fa

## 4. Validate and Export
Check how well the model performs and export it for deployment.

In [4]:
metrics = model.val()  # evaluate model performance on the validation set

Ultralytics 8.4.0 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 73 layers, 3,008,768 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2012.1±887.1 MB/s, size: 80.1 KB)
val: Scanning /content/pothole-detection-2-6/valid/labels.cache... 4943 images, 18 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 4943/4943 1.2Git/s 0.0s
val: /content/pothole-detection-2-6/valid/images/1000_png_jpg.rf.acda5eefcf5eee544b8c204c3be59d6c.jpg: 1 duplicate labels removed
val: /content/pothole-detection-2-6/valid/images/pic-235-_jpg.rf.f67f466b6adc36864093db4e7cd7e535.jpg: 1 duplicate labels removed
val: /content/pothole-detection-2-6/valid/images/pic-80-_jpg.rf.f01bb335f2719d722c93f6c074cb68a8.jpg: 1 duplicate labels removed
val: /content/pothole-detection-2-6/valid/images/youtube-40_jpg.rf.7052f2392445f8e3276ac5f62d2270e6.jpg: 1 duplicate labels removed
WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 9497, len(

In [5]:
# Download the trained weights
from google.colab import files
try:
  print("Attempting to download best.pt...")
  files.download('/content/runs/detect/train/weights/best.pt')
except:
  print("Automatic download failed. Please manually download: /content/runs/detect/train/weights/best.pt")

Attempting to download best.pt...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>